In [4]:
import os
import urllib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

In [5]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC'
os.chdir(path)

csv = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\csv.csv'
f = open(csv,'w+')

txt = (r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\data_exploration\daily' + 
       r'\akr02.txt')
g = open(txt,'w+')

empty_line = ('\n')

xl_file = (path + r'\meta_files\colocation_metadata.xlsx')

In [6]:
xl = pd.ExcelFile(xl_file)

meta = xl.parse('metadata')
meta['sdate'] = meta.sdate.dt.date
meta['edate'] = meta.edate.dt.date
meta = meta.drop(meta.columns[3],axis=1)

In [ ]:
sns.set_style('dark')
sns.set(rc={'figure.figsize' : (11.7,8.27)})
n = 0
for index,rows in meta.iterrows():
    try:
        print('Pulling data from ' + str(meta.ID[n]) + '...')
        urllib.request.urlretrieve(
            'http://coagmet.colostate.edu/cig-bin/web_services.pl?type=daily&sids=' +
            str(meta.ID[n]) + '&sdate=' + str(meta.sdate[n]) + '&edate=' + str(meta.edate[n]) + 
            '&elems=tave,tmax,tmin,vp,rhmax,rhmin,sr,wrun,pp,st5mx,st5mn,st15mx,' + 
            'st15mn,volts,gust,gustdir', filename=csv)
    except Exception as e:
        print('Something went wrong while pullin the data for ' + str(meta.ID[n]) + ': ' + e.args[0])
        print(empty_line)
    else:
        print('Data successfully pulled from ' + str(meta.ID[n]))
        print(empty_line)
        
    try:
        print('Loading data from ' + str(meta.ID[n]) + ' into a pandas dataframe...')
        df = pd.read_csv(csv)
        df = df.reset_index()
    except Exception as e:
        print('Something went wrong while loading data from ' + str(meta.ID[n]) + 
             'a pandas dataframe: ' + e.args[0])
        print(empty_line)
    else:
        print('Data from ' + str(meta.ID[n]) + ' succssfully loaded into a pandas dataframe')
        print(empty_line)
        
    try:
        print('Preparing the ' + str(meta.ID[n]) + ' for analysis...')
        df.columns = ['station','date','tave','tmax','tmin','vp','rhmax','rhmin','sr','wrun','pp','st5mx',
             'st5mn','st15mx','st15mn','volts','gust','gustdir']
        df['date'] = pd.to_datetime(df.date)
        df.drop(df.tail(1).index, inplace=True)
        cols = df.columns.drop(['station','date'])
        df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    except Exception as e:
        print('Something went wrong while preparing the ' + str(meta.ID[n]) + ' for analysis.')
        print(empty_line)
    else:
        print('The ' + str(meta.ID[n]) + ' is ready for analysis.')
        print(empty_line)
        
    try:
        print('Writing a statistical summary for of the raw data from' + str(meta.ID[n]) + '...')
        txt = (r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\data_exploration\daily' + 
        r'\nullreport_' + str(meta.ID[n]) + '.txt')
        g = open(txt,'w+') 
        title = ('Summary of data from ' + str(meta.ID[n]) + 
                 ' for the entire period of record:')
        g.write(title.center(24))
        g.write(empty_line)
        g.write(empty_line)
        g.write('Statistical Summary of Raw Data:')
        g.write(empty_line)
        g.write(empty_line)
        raw_summary = df.describe()
        g.write(raw_summary + empty_line)
        g.write(empty_line)
        og_lenght = len(df.index)
        print('The raw dataset from ' + str(meta.ID[n]) + ' contains ' + str(og_lenght) + 
              ' observations.')
        print(empty_line)
    except Exception as e:
        print('Something went wrong while writing the raw summary for ' + str(meta.ID[n]) + 
             ' to a text file: ' + e.args[0])
    else:
        print('Text file for ' + str(meta.ID[n]) + 'successfully created appended.')
        
    try:
        print('Checking for null values reported in the average daily temperature column...')
        null_obs_string = 'Summary of null observations found in the raw dataset:'
        g.write(null_obs_string)
        g.write(empty_line)
        d = 0
        tave_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['tave'][d])):
                tave_nulls = tave_nulls + 1
            d = d + 1
        tave_string = ('There are ' + str(tave_nulls) + 
                       ' null observations of daily average temperature in the dataset.')
        g.write(tave_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             ' average temperature column from ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('Tave check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
              ' in the daily maximum temperature column...')
        d = 0 
        tmax_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['tmax'][d])):
                tmax_nulls = tmax_nulls + 1
            d = d + 1
        tmax_string = ('There are ' + str(tmax_nulls) + 
                       ' null observations of daily maximum temperature in the dataset.')
        g.write(tmax_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             'maximum temperature column from ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('Tmax check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
             ' in the daily minimum temperature column...')
        d = 0
        tmin_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['tmin'][d])):
                tmin_nulls = tmin_nulls + 1
            d = d + 1
        tmin_string = ('There are ' + str(tmin_nulls) + 
                       ' null observations of daily minimum temperature in the dataset')
        g.write(tmin_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' +
             'minimum temperature column from ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('Tmin check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
             ' in the daily average vapor pressure column...')
        d = 0
        vp_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['vp'][d])):
                vp_nulls = vp_nulls + 1
            d = d + 1
        vp_string = ('There are ' + str(vp_nulls) + 
                     ' null observations of daily average vapor pressure in the dataset.')
        g.write(vp_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             'average vapor pressure column from ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('vp check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
              ' in the daily maximum relative humidity column...')
        d = 0
        rhmax_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['rhmax'][d])):
                rhmax_nulls = rhmax_nulls + 1
            d = d + 1 
        rhmax_string = ('There are ' + str(rhmax_nulls) + 
                       ' null observations of daily maximum relative humidity in the dataset.')
        g.write(rhmax_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             ' maximum relative humidity column from ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('rhmax check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
             ' in the daily minimum relative humidity column...')
        d = 0
        rhmin_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['rhmin'][d])):
                rhmin_nulls = rhmin_nulls + 1
            d = d + 1 
        rhmin_string = ('There are ' + str(rhmin_nulls) + 
                        ' null observations of daily minimum relative humidity in the dataset.')
        g.write(rhmin_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             ' minimum relative humidity column for ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('rhmin check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) +
             ' in the daily average solar radiation column...')
        d = 0
        sr_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['rhmin'][d])):
                sr_nulls = sr_nulls + 1
            d = d+1
        sr_string = ('There are ' + str(sr_nulls) + 
                    ' null observations of daily average solar radiation in the dataset.')
        g.write(sr_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             'average solar radiation column for ' + str(meta.ID[n]) + ': ' + e.args[0])
        
        
        
        
        
        
    
        